In [1]:
import tensorflow as tf
from msalib import layers
from msalib import network
from msalib import train
from msalib.utils import load_dataset
import yaml


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [4]:
def loss_func(output, label):
    """Loss function (softmax cross entropy)

    Arguments:
        output {tf tensor} -- output from network
        label {tf tensor} -- labels

    Returns:
        tf tensor -- loss
    """

    return tf.losses.softmax_cross_entropy(
        logits=output, onehot_labels=label)


def run_mnist(config):
    """Run mnist/fashion_mnist classification

    Arguments:
        config {dict} -- read from config.yml
    """

    print('='*100)
    print('Run CNN on MNIST')
    print('='*100)
    print('Config:')
    for key, value in config.items():
        print('{:20} ({})'.format(key, value))
    print('='*100)

    tf.logging.set_verbosity(
        getattr(tf.logging, config['verbosity']))
    tf.set_random_seed(config['seed'])

    # Generate data
    trainset, testset = load_dataset(
        name=config['dataset_name'],
        num_train=config['num_train'],
        num_test=config['num_test'],
        lift_dim=config['lift_dimension']
    )

    # Build network
    input = tf.placeholder(
        tf.float32, [None, 28, 28, config['lift_dimension']], name='input')
    output = tf.placeholder(tf.float32, [None, 10], name='output')
    net = network.Network(name='msa_net')

    net.add(layers.Conv2D(
        input_shape=input.shape[1:],
        filters=config['num_channels'],
        kernel_size=config['filter_size'],
        padding=config['padding'],
        activation=config['activation'],
        msa_rho=config['rho'],
        msa_reg=config['reg'],
        name='conv2d_0')
    )

    net.add(layers.AveragePooling2D(
        pool_size=2, strides=2, padding=config['padding'], name='avg_pool_0')
    )

    net.add(layers.Conv2D(
        input_shape=input.shape[1:],
        filters=config['num_channels'],
        kernel_size=config['filter_size'],
        padding=config['padding'],
        activation=config['activation'],
        msa_rho=config['rho'],
        msa_reg=config['reg'],
        name='conv2d_init')
    )

    net.add(layers.AveragePooling2D(
        pool_size=2, strides=2, padding=config['padding'], name='avg_pool_1')
    )

    for n in range(config['num_layers']):
        net.add(layers.ResidualConv2D(
            filters=config['num_channels'],
            kernel_size=config['filter_size'],
            padding=config['padding'],
            activation=config['activation'],
            msa_rho=config['rho'],
            msa_reg=config['reg'],
            delta=config['delta'],
            name='residualconv2d_{}'.format(n)))

    net.add(layers.Lower(name='lower'))
    net.add(layers.Flatten(name='flatten'))
    net.add(layers.Dense(
        units=10, msa_rho=config['rho'], msa_reg=config['reg'], name='dense'))

    net.msa_compute_x(input)
    net.msa_compute_p(output, loss_func)
    net.summary()
    sess = tf.Session()

    # MSA trainer
    msa_trainer = train.MSATrainer(
        network=net,
        name='MSA_trainer',
        maxiter=config['msa_maxiter'],
        perturb_init=config['msa_perturb_init'])
    msa_trainer.initialize(sess)
    msa_trainer.train(
        session=sess,
        trainset=trainset,
        testset=testset,
        batch_size=config['batch_size'],
        num_epochs=config['num_epochs'],
        buffer_size=config['buffer_size'],
        print_step=config['print_step'])
    '''
    # SGD trainer
    sgd_trainer = train.BPTrainer(
        network=net, name='SGD_trainer',
        method='GradientDescentOptimizer',
        args={'learning_rate': config['sgd_lr']})
    sgd_trainer.initialize(sess)
    sgd_trainer.train(
        session=sess,
        trainset=trainset,
        testset=testset,
        batch_size=config['batch_size'],
        num_epochs=config['num_epochs'],
        buffer_size=config['buffer_size'],
        print_step=config['print_step'])

    # Adagrad trainer
    adagrad_trainer = train.BPTrainer(
        network=net, name='Adagrad_trainer',
        method='AdagradOptimizer',
        args={'learning_rate': config['adagrad_lr']})
    adagrad_trainer.initialize(sess)
    adagrad_trainer.train(
        session=sess,
        trainset=trainset,
        testset=testset,
        batch_size=config['batch_size'],
        num_epochs=config['num_epochs'],
        buffer_size=config['buffer_size'],
        print_step=config['print_step'])

    # Adam trainer
    Adam_trainer = train.BPTrainer(
        network=net, name='Adam_trainer',
        method='AdamOptimizer',
        args={'learning_rate': config['adam_lr']})
    Adam_trainer.initialize(sess)
    Adam_trainer.train(
        session=sess,
        trainset=trainset,
        testset=testset,
        batch_size=config['batch_size'],
        num_epochs=config['num_epochs'],
        buffer_size=config['buffer_size'],
        print_step=config['print_step'])
      '''

In [3]:
!python3 -u main_mnist.py | tee mnist.log


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Run CNN on MNIST
Config:
dataset_name         (mnist)
seed                 (1)
verbosity            (WARN)
print_step           (100)
lift_dimension       (1)
num_train            (60000)
num_test             (10000)
num_channels         (5)
filter_size          (3)
padding              (same)
num_layers           (2)
rho                  (0.03)
reg                  (0.001)
activation           (tanh)
delta                (0.5)
batch_size           (128)
buffer_size          (512)
num_epochs           (1)
sgd_lr               (0.01)
adagrad_lr           (0.01)
adam_lr              (0.001)
msa_maxiter          (10)
msa_perturb_init     (0.0)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast inst

Step   225 of   469: 
Train loss: 0.4957673425896693
Test loss: 0.4958732517088874
Step   230 of   469: 
Train loss: 0.49451214807518457
Test loss: 0.49457962204844264
Step   235 of   469: 
Train loss: 0.4921166934704376
Test loss: 0.4927618523775521
Step   240 of   469: 
Train loss: 0.4926631066758754
Test loss: 0.4924122989177704
Step   245 of   469: 
Train loss: 0.49086415515107623
Test loss: 0.4904660663362277
Step   250 of   469: 
Train loss: 0.4899554462251017
Test loss: 0.4909431474693751
Step   255 of   469: 
Train loss: 0.4888014793395996
Test loss: 0.4894995843454943
Step   260 of   469: 
Train loss: 0.4867824760533996
Test loss: 0.48697429668095155
Step   265 of   469: 
Train loss: 0.48523149753020983
Test loss: 0.485996803489782
Step   270 of   469: 
Train loss: 0.48493501392461485
Test loss: 0.485340859677832
Step   275 of   469: 
Train loss: 0.48269837690612016
Test loss: 0.4830962687225665
Step   280 of   469: 
Train loss: 0.48278380273762395
Test loss: 0.483016562411340

Step   225 of   469: 
Train loss: 2.2524853197194763
Test loss: 2.2529329283762785
Step   230 of   469: 
Train loss: 2.2511534711061896
Test loss: 2.251514822749768
Step   235 of   469: 
Train loss: 2.249378614506479
Test loss: 2.2496854733612577
Step   240 of   469: 
Train loss: 2.2476421796669395
Test loss: 2.2479893777330044
Step   245 of   469: 
Train loss: 2.2456171775268294
Test loss: 2.2459436412584983
Step   250 of   469: 
Train loss: 2.24367604417316
Test loss: 2.2440550327301025
Step   255 of   469: 
Train loss: 2.241596500752336
Test loss: 2.2421015985941484
Step   260 of   469: 
Train loss: 2.239576121508065
Test loss: 2.2398870819706027
Step   265 of   469: 
Train loss: 2.2373437881469727
Test loss: 2.237594143819001
Step   270 of   469: 
Train loss: 2.2353461617130344
Test loss: 2.2357260291859253
Step   275 of   469: 
Train loss: 2.2334859290365445
Test loss: 2.2338335473658675
Step   280 of   469: 
Train loss: 2.231290724317906
Test loss: 2.2316262418940918
Step   285 o

In [3]:
if __name__ == "__main__":
    config = yaml.safe_load(open("config.yml"))
    run_mnist(config['mnist'])

Run CNN on MNIST
Config:
dataset_name         (mnist)
seed                 (1)
verbosity            (WARN)
print_step           (100)
lift_dimension       (1)
num_train            (60000)
num_test             (10000)
num_channels         (5)
filter_size          (3)
padding              (same)
num_layers           (2)
rho                  (0.03)
reg                  (0.001)
activation           (tanh)
delta                (0.5)
batch_size           (128)
buffer_size          (512)
num_epochs           (1)
sgd_lr               (0.01)
adagrad_lr           (0.01)
adam_lr              (0.001)
msa_maxiter          (10)
msa_perturb_init     (0.0)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_0 (Conv2D)            (None, 28, 28, 5)     

KeyboardInterrupt: 